In [10]:
cd "/Users/sylviathsu/Documents/COS243/LocalAILibrarian"

/Users/sylviathsu/Documents/COS243/LocalAILibrarian


In [11]:
!pip install -q \
  llama-index \
  EbookLib \
  html2text \
  gradio \
  llama-index-embeddings-huggingface \
  llama-index-llms-ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
import os
import logging
import gradio as gr
from pathlib import Path
import shutil
from typing import List, Tuple
import json
from datetime import datetime

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    Settings
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter

In [14]:
# Initialize embedding model globally
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Set the embedding model as default
Settings.embed_model = embed_model

def setup_logging():
    """Configure logging settings"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

logger = setup_logging()

def process_documents(doc_folder: str):
    """Process documents from the specified folder"""
    try:
        reader = SimpleDirectoryReader(
            input_dir=doc_folder,
            recursive=True,
            filename_as_id=True,
            required_exts=[".txt", ".epub", ".pdf"]
        )
        return reader.load_data()
    except Exception as e:
        logger.error(f"Document processing error: {e}")
        raise

def generate_embeddings(documents):
    """Generate embeddings and create vector store index"""
    try:
        # Configure text splitting
        splitter = SentenceSplitter(
            chunk_size=512,
            chunk_overlap=50,
            paragraph_separator="\n\n",
            include_metadata=True
        )
        
        # Create and save index using the global embedding model
        vector_index = VectorStoreIndex.from_documents(
            documents,
            embed_model=embed_model,  # Explicitly pass the embedding model
            transformations=[splitter],
            show_progress=True
        )
        
        vector_index.storage_context.persist(persist_dir="storage")
        return vector_index
    except Exception as e:
        logger.error(f"Embedding generation error: {e}")
        raise

def configure_query_engine():
    """Configure the query engine with optimized parameters"""
    try:
        # Configure LLM with increased timeout
        llm = Ollama(
            model="phi3.5:3.8b-mini-instruct-q4_K_M",
            request_timeout=120.0,
            temperature=0.1
        )
        
        # Load index with explicit embedding model
        storage_context = StorageContext.from_defaults(persist_dir="storage")
        index = load_index_from_storage(
            storage_context,
            embed_model=embed_model  # Explicitly pass the embedding model
        )
        
        query_engine = index.as_query_engine(
            llm=llm,
            similarity_top_k=3,
            response_mode="compact",
            streaming=False
        )
        
        return query_engine
    except Exception as e:
        logger.error(f"Query engine configuration error: {e}")
        raise

def save_conversation_history(history: List[Tuple[str, str]], file_format: str = "txt") -> str:
    """Save conversation history to a file"""
    try:
        # Create exports directory if it doesn't exist
        export_dir = Path("exports")
        export_dir.mkdir(exist_ok=True)
        
        # Generate timestamp for filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        if file_format == "json":
            # Save as JSON
            filename = export_dir / f"conversation_{timestamp}.json"
            conversation_data = {
                "timestamp": timestamp,
                "messages": [{"question": q, "answer": a} for q, a in history]
            }
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(conversation_data, f, indent=2, ensure_ascii=False)
        else:
            # Save as plain text
            filename = export_dir / f"conversation_{timestamp}.txt"
            with open(filename, 'w', encoding='utf-8') as f:
                for q, a in history:
                    f.write(f"Q: {q}\n")
                    f.write(f"A: {a}\n")
                    f.write("-" * 80 + "\n")
        
        return f"Conversation saved to {filename}"
    except Exception as e:
        logger.error(f"Error saving conversation: {e}")
        return f"Error saving conversation: {str(e)}"

def create_gradio_interface():
    """Create and configure the Gradio interface"""
    # Initialize query engine at the start
    global_query_engine = configure_query_engine() if Path("storage").exists() else None
    
    def handle_file_upload(files, progress=gr.Progress()):
        """Handle file upload and index update"""
        try:
            nonlocal global_query_engine
            
            # Create library directory if it doesn't exist
            Path("library").mkdir(exist_ok=True)
            
            # Copy uploaded files to library directory
            progress(0, desc="Copying files...")
            for file in files:
                shutil.copy2(file.name, "library")
            
            # Process documents and update index
            progress(0.3, desc="Processing documents...")
            documents = process_documents("library")
            
            progress(0.6, desc="Generating embeddings...")
            generate_embeddings(documents)
            
            progress(0.9, desc="Configuring query engine...")
            global_query_engine = configure_query_engine()
            
            progress(1.0, desc="Done!")
            return "Files uploaded and indexed successfully!"
        except Exception as e:
            logger.error(f"Upload error: {e}")
            return f"Error processing files: {str(e)}"

    def query_docs(query: str, history: List[Tuple[str, str]]) -> Tuple[str, List]:
        """Process queries and update conversation history"""
        nonlocal global_query_engine
        
        if not query.strip():
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
        
        if global_query_engine is None:
            error_msg = "Please upload documents first or wait for indexing to complete."
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
        
        try:
            response = global_query_engine.query(query)
            
            # Extract source information
            sources = []
            if hasattr(response, 'source_nodes'):
                sources = [
                    f"- {node.node.metadata.get('file_name', 'Unknown')}"
                    for node in response.source_nodes
                ]
            
            source_text = "\nSources:\n" + "\n".join(sources) if sources else ""
            full_response = str(response) + source_text
            
            history.append((query, full_response))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
            
        except Exception as e:
            logger.error(f"Query processing error: {e}")
            error_msg = f"Error processing query: {str(e)}"
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history

    # Create the Gradio interface
    with gr.Blocks(title="Local AI Librarian") as app:
        gr.Markdown("# Local AI Librarian")
        gr.Markdown("Upload documents and search through them using natural language queries.")
        
        # Document upload section
        with gr.Row():
            with gr.Column():
                file_output = gr.Textbox(label="Upload Status")
                upload_button = gr.File(
                    file_count="multiple",
                    label="Upload Documents",
                    file_types=[".txt", ".pdf", ".epub"]
                )
        
        # Query section
        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="Enter your query",
                    placeholder="What would you like to know about your documents?",
                    lines=2
                )
                examples = gr.Examples(
                    examples=[
                        "What are the main themes in the documents?",
                        "Summarize the key points about...",
                        "Find relevant passages about..."
                    ],
                    inputs=query_input
                )
        
        # Control buttons
        with gr.Row():
            submit_btn = gr.Button("Search")
            clear_btn = gr.Button("Clear History")
            
        # Export controls
        with gr.Row():
            export_format = gr.Radio(
                choices=["txt", "json"],
                value="txt",
                label="Export Format"
            )
            export_btn = gr.Button("Export Conversation")
            export_status = gr.Textbox(label="Export Status")
            
        # Chat history and output
        chat_history = gr.State([])
        output = gr.Textbox(
            label="Results",
            lines=20,
            autoscroll=False
        )
        
        # Set up event handlers
        upload_button.upload(
            handle_file_upload,
            inputs=[upload_button],
            outputs=[file_output]
        )
        
        submit_btn.click(
            query_docs,
            inputs=[query_input, chat_history],
            outputs=[output, chat_history]
        )
        
        query_input.submit(
            query_docs,
            inputs=[query_input, chat_history],
            outputs=[output, chat_history]
        )
        
        clear_btn.click(
            lambda: ([], []),
            outputs=[output, chat_history]
        )
        
        export_btn.click(
            save_conversation_history,
            inputs=[chat_history, export_format],
            outputs=[export_status]
        )
        
    return app

if __name__ == "__main__":
    try:
        logger.info("Starting application...")
        
        # Create necessary directories
        Path("library").mkdir(exist_ok=True)
        Path("storage").mkdir(exist_ok=True)
        Path("exports").mkdir(exist_ok=True)
        
        # Launch Gradio interface
        logger.info("Launching Gradio interface...")
        app = create_gradio_interface()
        app.launch(share=False)
        
    except Exception as e:
        logger.error(f"Application error: {e}")
        raise

2024-12-24 18:58:36,274 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2024-12-24 18:58:37,946 - sentence_transformers.SentenceTransformer - INFO - 2 prompts are loaded, with the keys: ['query', 'text']
2024-12-24 18:58:37,953 - __main__ - INFO - Starting application...
2024-12-24 18:58:37,953 - __main__ - INFO - Launching Gradio interface...
2024-12-24 18:58:38,795 - llama_index.core.indices.loading - INFO - Loading all indices.
2024-12-24 18:58:38,876 - httpx - INFO - HTTP Request: GET http://127.0.0.1:7860/startup-events "HTTP/1.1 200 OK"
2024-12-24 18:58:38,885 - httpx - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


2024-12-24 18:58:39,327 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
Generating embeddings: 100%|██████████| 662/662 [00:54<00:00, 12.24it/s]
2024-12-24 18:59:58,775 - llama_index.core.indices.loading - INFO - Loading all indices.
Batches: 100%|██████████| 1/1 [00:00<00:00,  1.08it/s]
2024-12-24 19:02:12,628 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:05<00:00,  5.39s/it]
2024-12-24 19:03:13,009 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
Batches: 100%|██████████| 1/1 [00:05<00:00,  5.94s/it]
2024-12-24 19:04:41,742 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
